In [3]:
from operator import truediv

#1.导入库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
rcParams['font.sans-serif'] = ['SimHei']
rcParams['font.sans-serif'] = ['STHeiti']

In [4]:
#2.导入数据
df = pd.read_csv('/Users/zhangke/Downloads/Data Analyst/house_sales.csv')

In [5]:
#3.数据概览
print('总记录数',len(df))
print('字段数量',len(df.columns))
df.head(5)
df.info()

总记录数 106118
字段数量 12
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106118 entries, 0 to 106117
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   city        106118 non-null  object
 1   address     104452 non-null  object
 2   area        105324 non-null  object
 3   floor       104024 non-null  object
 4   name        105564 non-null  object
 5   price       105564 non-null  object
 6   province    106118 non-null  object
 7   rooms       104036 non-null  object
 8   toward      105240 non-null  object
 9   unit        105564 non-null  object
 10  year        57736 non-null   object
 11  origin_url  105564 non-null  object
dtypes: object(12)
memory usage: 9.7+ MB


In [6]:
# 4.数据清洗
df.head(5)
#删除无用的数据列
df.drop(columns='origin_url',inplace=True)

In [7]:
# 检查是否有缺失值
df.isna().sum()
#删除缺失值
df.dropna(inplace=True)

In [8]:
#检查是否有重复值
df.duplicated().sum()
#删除重复值
df.drop_duplicates(inplace=True)
print(len(df))

28104


In [9]:
#面积的数据类型转换
df['area'] = df['area'].str.replace('㎡','').astype(float)
#售价的数据类型转换
df['price'] = df['price'].str.replace('万','').astype(float)
# 朝向的数据类型转换
#df['toward'].value_counts()
df['toward'] = df['toward'].astype('category')
df['unit'] = df['unit'].str.replace('元/㎡','').astype(float)
# 建造年份的数据类型
df['year'] = df['year'].str.replace('年建','').astype(int)

In [19]:
print(len(df))
len(df[(df['area']<600) & (df['area']>20)])

28079


28079

In [20]:
#异常值的处理
#房屋面积的异常处理
df = df[(df['area']<600) & (df['area']>20)]

#df.info()

In [23]:
#房屋售价的异常值处理 IQR
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
low_price = Q1 - 1.5 * IQR
high_price = Q3 + 1.5 * IQR
len(df[( df['price']<high_price) & (df['price']>low_price) ])

26135

In [36]:
#5.新数据特征构造
# 地区district
df['district'] = df['address'].str.split('-').str[0]

In [49]:
# 楼层的类型floor_type
#df.sample(10)
df['floor_type'] = df['floor'].str.split('（').str[0].astype('category')
def fun1(str1):
    if pd.isna(str1):
        return '未知'
    elif '低' in str1:
        return '低楼层'
    elif '中' in str1:
        return '中楼层'
    elif '高' in str1:
        return '高楼层'
df['floor_type2'] = df['floor'].apply(fun1).astype('category')
# 是否是直辖市 zxs
# def fun2(str2):
#     if str2 in ['北京','上海','天津','重庆']:
#         return True
#     else:
#         return False
# df['city'].apply(fun2)

df['city'].apply(lambda x:1 if x in ['北京','上海','天津','重庆'] else 0)
# 卧室的数量 bedrooms
df['bedrooms'] = df['rooms'].str.split('室').str[0].astype(int)
# 客厅的数量 living rooms
# df['livingrooms'].str.split('室').str[1].str.split('厅').str[0].astype(int) #有点太麻烦了,用正则表达式
df['livingrooms'] = df['rooms'].str.extract(r'(\d+)厅').astype(int)
# 楼龄building_age
df['building_age'] = 2025 - df['year']


In [70]:


# 价格的分段 price_lables
pd.cut(df['price'], bins=4, labels=['低价','中价','高价','豪华'])

0         低价
2         低价
4         低价
6         低价
8         低价
          ..
106080    低价
106082    低价
106084    低价
106086    低价
106088    低价
Name: price, Length: 28079, dtype: category
Categories (4, object): ['低价' < '中价' < '高价' < '豪华']

In [69]:
df.sample(10)

,city,address,area,floor,name,price,province,rooms,toward,unit,year,district,floor_type,floor_type2,bedrooms,livingrooms,building_age
15346,招远,牟平中心区,92.00,中层（共6层）,领海公馆,77.5,山东,2室2厅,南北向,8424.0,2016,牟平中心区,中层,中楼层,2,2,9
71688,罗源,铜盘路,83.00,中层（共7层）,泉塘小区三区,163.0,福建,3室2厅,西南向,19639.0,1998,铜盘路,中层,中楼层,3,2,27
29286,百色,龙景区,128.00,高层（共33层）,三祺·城光中心,106.0,广西,3室2厅,南向,8281.0,2017,龙景区,高层,高楼层,3,2,8
79824,仙桃,市区,110.33,高层（共24层）,元泰未来城,68.0,湖北,3室2厅,南北向,6163.0,2015,市区,高层,高楼层,3,2,10
15658,玉林,江南公园,126.90,高层（共29层）,华商国际欧洲城,69.0,广西,4室2厅,南向,5437.0,2016,江南公园,高层,高楼层,4,2,9
100730,盐城,亭湖,130.00,中层（共20层）,奥体玫瑰园,139.8,江苏,3室2厅,南北向,10754.0,2015,亭湖,中层,中楼层,3,2,10
372,上海,古北,114.05,低层（共18层）,古北嘉年华庭,1000.0,直辖市,2室2厅,南向,87681.0,2005,古北,低层,低楼层,2,2,20
8912,青岛,夏庄,80.00,高层（共11层）,鑫江玫瑰园,98.0,山东,2室1厅,南北向,12250.0,2021,夏庄,高层,高楼层,2,1,4
9184,丽水,城西括苍路,94.00,高层（共20层）,绿城留香园,248.0,浙江,3室2厅,南北向,26383.0,2019,城西括苍路,高层,高楼层,3,2,6
100936,永登,东部市场,92.00,低层（共30层）,兰新万国港,120.0,甘肃,2室1厅,西南向,13043.0,2013,东部市场,低层,低楼层,2,1,12


In [51]:



df.info()
#6.问题分析及可视化


<class 'pandas.core.frame.DataFrame'>
Index: 28079 entries, 0 to 106088
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   city         28079 non-null  object  
 1   address      28079 non-null  object  
 2   area         28079 non-null  float64 
 3   floor        28079 non-null  object  
 4   name         28079 non-null  object  
 5   price        28079 non-null  float64 
 6   province     28079 non-null  object  
 7   rooms        28079 non-null  object  
 8   toward       28079 non-null  category
 9   unit         28079 non-null  float64 
 10  year         28079 non-null  int64   
 11  district     28079 non-null  object  
 12  floor_type   28079 non-null  category
 13  floor_type2  28079 non-null  category
dtypes: category(3), float64(3), int64(1), object(7)
memory usage: 3.7+ MB
